In [2]:
import pandas as pd

**Задание 1**  
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [6]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
def define_rating_class(rating):
    if rating <= 2:
        return 'low'
    elif rating <= 4:
        return 'mid'
    elif rating in [4.5, 5.0]:
        return 'high'
    else:
        return 'maybe high'

In [8]:
ratings['class'] = ratings['rating'].apply(define_rating_class)
ratings.head()

# ratings['class'].value_counts()

,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,mid
1,1,1029,3.0,1260759179,mid
2,1,1061,3.0,1260759182,mid
3,1,1129,2.0,1260759185,low
4,1,1172,4.0,1260759205,mid


**Задание 2**  
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

  'Центр': ['москва', 'тула', 'ярославль'],  
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],  
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']  
}

Результат классификации запишите в отдельный столбец region.

In [47]:
import pandas as pd
keywords = pd.read_csv('keywords.csv')
keywords.head(2)

,keyword,shows
0,вк,64292779
1,одноклассники,63810309


In [10]:
geo_data = {
        'Центр': ['москва', 'тула', 'ярославль'],
        'Северо-Запад': ['петербург', 'псков', 'мурманск'],
        'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
    }

In [11]:
def define_region(string_in, geo_data_dict):
    string_in = string_in.lower()
    for key, cities in geo_data_dict.items():
        for city in cities:
            if city in string_in:
                return key
    return 'undefined'

In [48]:
keywords['region'] = keywords['keyword'].apply(define_region, geo_data_dict=geo_data)
keywords.head()
keywords.region.value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 3 (бонусное)**  
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.
2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [49]:
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
years = list(range(1950, 2011))
years[:10]

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959]

In [50]:
def production_year(title):
    for year in years:
        if str(year) in title:
            return year
    return 1900

In [51]:
movies['year'] = movies['title'].apply(production_year)
movies.head(2)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995


In [65]:
movies_ratings = pd.merge(movies, ratings, how='left', on='movieId')
movies_ratings.groupby('year')['rating'].mean().sort_values(ascending=False).head(60)
# итого, невооруженным глазом видно, что таки да... а если точнее, то рейтинги выше у старых фильмов

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
1974    3.978704
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
1959    3.841033
1958    3.823171
1975    3.823077
1969    3.817376
1967    3.814516
1960    3.809211
1976    3.806794
1973    3.806667
1979    3.777966
1981    3.758763
1953    3.754587
1955    3.751724
1971    3.748830
1968    3.737844
1966    3.736726
1980    3.708816
1963    3.705966
1970    3.700000
1965    3.678715
1961    3.663851
1983    3.657635
1984    3.651486
1982    3.638306
1991    3.628265
1956    3.617647
1987    3.606015
1900    3.602960
1985    3.588998
1988    3.552990
1978    3.549003
2007    3.548538
1994    3.537963
2010    3.537157
1993    3.536664
1989    3.534349
1986    3.528265
2009    3.520378
2006    3.511478
2008    3.498057
2004    3.491490
1998    3.472754
1995    3.472118
1999    3.471692
1992    3.466754
2001    3.460052
2002    3.456297
1990    3.449169
2005    3.448434
2003    3

In [66]:
# усредним среднее по десятилетиям
years_rating = movies_ratings.groupby('year')['rating'].mean().reset_index()
years_rating['decade'] = years_rating['year'].apply(lambda year: year // 10 * 10)
years_rating.groupby('decade')['rating'].mean()

decade
1900    3.602960
1950    3.869542
1960    3.775814
1970    3.810796
1980    3.622562
1990    3.487774
2000    3.473560
2010    3.537157
Name: rating, dtype: float64